In [3]:
import sys
sys.path.append('..')

from utils.spark_session import get_spark
spark = get_spark()

# Analysis
### Answering possible business questions

In [4]:
category_df = spark.read.parquet(
    '../data/gold/category', 
    )

products_df = spark.read.parquet(
    '../data/gold/products',
    )

sales_df = spark.read.parquet(
    '../data/gold/sales',
    )

stores_df = spark.read.parquet(
    '../data/gold/store',
    )

warranty_df = spark.read.parquet(
    '../data/gold/warranty',
    )

#### Q1) Which product categories generate the most revenue, and how consistent is that revenue?

In [6]:
from pyspark.sql.functions import broadcast
from pyspark.sql import functions as F

cat_revenue = (
    sales_df
    .join(broadcast(products_df), "product_id")
    .join(broadcast(category_df), "category_id")
    .groupBy("category_id", "category_name")
    .agg(
        F.sum("total_revenue").alias("total_revenue"),
        F.avg("total_revenue").alias("avg_txn_revenue"),
        F.stddev("total_revenue").alias("revenue_volatility")
    )
    .orderBy(F.desc("total_revenue"))
)

cat_revenue.show()


+-----------+--------------------+--------------+------------------+------------------+
|category_id|       category_name| total_revenue|   avg_txn_revenue|revenue_volatility|
+-----------+--------------------+--------------+------------------+------------------+
|      cat-4|          smartphone|1.0842158395E9|  8563.15920435339|50714.512429496994|
|      cat-3|              tablet| 1.038578306E9|11798.272208842642| 67974.99582918423|
|      cat-2|               audio| 9.856494305E8| 9184.381282729832| 55739.98457567599|
|     cat-10|         accessories|  8.76624984E8| 8169.926877230915|  48151.3680311925|
|      cat-5|            wearable|  7.90606623E8|10086.326584507042|58345.501885786456|
|      cat-7|             desktop|  6.41002078E8| 6559.109335188842|  39719.0660995124|
|      cat-8|subscription service|  4.25408187E8| 7294.128922190597| 43583.22159585385|
|      cat-6|    streaming device|  2.43220899E8|  8320.08001231485| 50231.37390684874|
|      cat-9|       smart speake

#### Key insights

* Smartphones dominate revenue

  * Highest total revenue → core business pillar

* Tablets have higher avg transaction value

  * Fewer purchases, but larger baskets

* Accessories generate high revenue with lower volatility

  * Very stable, predictable revenue stream

* Subscription services are mid-revenue but stable

  * Likely recurring behavior

### Q2) Which stores are top performers within a region, for example Asia-Pacific(APAC), and are they revenue outliers?

In [4]:
region_stats = (
    stores_df
    .groupBy("store_region")
    .agg(F.avg("total_store_revenue").alias("avg_region_revenue"))
)

store_perf = (
    stores_df
    .join(region_stats, "store_region")
    .withColumn(
        "revenue_vs_region_avg",
        F.col("total_store_revenue") - F.col("avg_region_revenue")
    )
    .orderBy("store_region", "rank_by_region")
)

store_perf.filter(F.col('store_region').like('APAC')).show()


+------------+--------+-------------------+----------+-----------+-------------------+--------------+------------------+---------------------+
|store_region|store_id|         store_name|      city|    country|total_store_revenue|rank_by_region|avg_region_revenue|revenue_vs_region_avg|
+------------+--------+-------------------+----------+-----------+-------------------+--------------+------------------+---------------------+
|        APAC|   st-37|     apple sanlitun|   beijing|      china|         9.772063E7|             1|     8.557383576E7| 1.2146794239999995E7|
|        APAC|   st-43|apple cotai central|     macau|      china|       9.61505555E7|             2|     8.557383576E7| 1.0576719739999995E7|
|        APAC|   st-74|apple central world|   bangkok|   thailand|       9.60897455E7|             3|     8.557383576E7| 1.0515909739999995E7|
|        APAC|   st-75|  apple beijing skp|   beijing|      china|       9.51214125E7|             4|     8.557383576E7|    9547576.739999995|

#### Insights

* Top APAC stores outperform the regional average by $10–12M

* There is a long tail of underperforming stores

* Even within the same city (e.g. Melbourne), store performance varies widely

#### Business takeaway

* Regional averages hide significant store-level variance

* High-performing stores could serve as operational benchmarks

* Underperformers may need:

  * Marketing intervention

  * Location analysis

  * Staffing / inventory adjustments

### Q3) Do newer products outperform older ones in revenue and volume?

In [5]:
age_perf = (
    sales_df
    .join(products_df, "product_id")
    .groupBy("product_age_yrs")
    .agg(
        F.sum("total_revenue").alias("total_revenue"),
        F.sum("quantity").alias("units_sold"),
        F.countDistinct("product_id").alias("num_products")
    )
    .orderBy("product_age_yrs")
)

age_perf.show()


+---------------+--------------+----------+------------+
|product_age_yrs| total_revenue|units_sold|num_products|
+---------------+--------------+----------+------------+
|           NULL|   3.0187431E8|    239545|           3|
|              2|  9.61379521E8|   1013162|          13|
|              3| 1.038416276E9|    954964|          12|
|              4|1.0452408755E9|    889132|          11|
|              5| 1.356196787E9|   1205385|          15|
|              6|1.4969564735E9|   1497561|          19|
+---------------+--------------+----------+------------+



#### Key Insights

* Revenue and sales increase with product maturity

* Products aged 5–6 years:

  * Highest total revenue

  * Highest unit sales

  * Largest product count

#### Important nuance

Newer products do NOT outperform older ones

Indicates:

* Strong brand loyalty

* Long product lifecycle

* Older models remain commercially relevant

#### Business implication

* Aggressive replacement cycles may not be necessary

* Focus on incremental upgrades vs full replacement

### Q4) Which products have the highest early-failure risk relative to sales volume?

In [6]:
sales_volume = (
    sales_df
    .groupBy("product_id")
    .agg(F.sum("quantity").alias("units_sold"))
)

claim_stats = (
    warranty_df
    .groupBy("product_id")
    .agg(
        F.sum("is_early_failure").alias("early_failures"),
        F.avg("days_to_claim").alias("avg_days_to_claim")
    )
)

failure_analysis = (
    sales_volume
    .join(claim_stats, "product_id")
    .withColumn(
        "early_failure_rate",
        F.col("early_failures") / F.col("units_sold")
    )
    .orderBy(F.desc("early_failure_rate"))
)

failure_analysis.show()


+----------+----------+--------------+-----------------+--------------------+
|product_id|units_sold|early_failures|avg_days_to_claim|  early_failure_rate|
+----------+----------+--------------+-----------------+--------------------+
|      p-71|     80068|            21|701.7436974789916|2.622770644951791E-4|
|      p-66|     70509|            18|705.5489361702128|2.552865591626601E-4|
|      p-83|     77098|            18|719.9606299212599|2.334690912864147E-4|
|      p-12|     77316|            18|697.3409090909091|2.328108024212323...|
|      p-70|     84337|            19|669.5767441860465|2.252866476161115...|
|      p-65|     78234|            17|          707.032| 2.17296827466319E-4|
|      p-32|     77443|            16|           682.52|2.066035664940666E-4|
|      p-56|     78135|            16|651.4378109452737|2.047737889550137...|
|      p-87|     74124|            15|714.6528301886792|2.023636069289299E-4|
|      p-78|     74933|            15| 720.109243697479|2.001788

#### Interpretation

* early_failure_rate is extremely low (≈0.02%)

  * Indicates strong overall product quality

* Products with high sales volume still appear at top

  * Rate-based analysis prevents false alarms

#### Key insight

* Early failures occur ~2 years after purchase

* Not immediate manufacturing defects

* Likely wear-and-tear or usage patterns

#### Business implication

* Warranty risk is manageable

* No urgent product recall signals

* Predictive maintenance programs could reduce claim volume

### Q5) How does revenue momentum evolve over time, and where do we see demand spikes?

In [ ]:
from pyspark.sql.window import Window
# Step 1: Aggregate daily revenue
daily_rev_df = (
    sales_df
    .filter(F.col("sale_date").isNotNull())
    .groupBy("sale_date")
    .agg(F.sum("total_revenue").alias("daily_revenue"))
    .orderBy("sale_date")
)

# Step 2: Compute rolling 7-day revenue
winspec = Window.orderBy("sale_date").rowsBetween(-6, 0)
daily_trend = daily_rev_df.withColumn(
    "rolling_7_day_revenue",
    F.sum("daily_revenue").over(winspec)
)

#step 3: Filtwring to get January 2020
y_2023 = daily_trend.filter(
    (F.col("sale_date") >= F.lit("2023-01-01")) &
    (F.col("sale_date") < F.lit("2024-01-01"))
)


In [11]:
import plotly.express as px

px.line(y_2023.toPandas(),x='sale_date', y=['daily_revenue','rolling_7_day_revenue'])

#### Interpretation

* Daily revenue is highly volatile (likely due to simulated data)

* Rolling 7-day revenue smooths noise

#### Key Insights

* Frequent demand spikes → promotion-driven sales

* Rolling average reveals underlying growth trend

* Ideal input for forecasting models